In [1]:
import pandas as pd
import numpy as np
import os 
from pathlib import Path

from datetime import datetime, timedelta
import time 
from dateutil.relativedelta import relativedelta

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import pyarrow.parquet as pq
import pyarrow as pa

In [2]:
import warnings
warnings.filterwarnings("ignore")

pd.options.display.max_rows=999
pd.options.display.max_columns=999

In [3]:
%%time
# train_file = r'/kaggle/input/amex-agg-data-rev2/agg_train_all_rev2_rev.parquet'
train_file = r'amex\agg_train_all_rev3.parquet'
df=pd.read_parquet(train_file, engine='pyarrow')

Wall time: 36.2 s


In [4]:
all_cols = df.columns.tolist()
all_cols.sort()
print(all_cols)
len(all_cols)

['B_10__first', 'B_10__last', 'B_10__last2max', 'B_10__last2min', 'B_10__max', 'B_10__mean', 'B_10__mean2std', 'B_10__min', 'B_10__range', 'B_10__std', 'B_11__first', 'B_11__last', 'B_11__last2max', 'B_11__last2min', 'B_11__last__log', 'B_11__max', 'B_11__mean', 'B_11__mean2std', 'B_11__mean__log', 'B_11__min', 'B_11__range', 'B_11__std', 'B_12__first', 'B_12__last', 'B_12__last2max', 'B_12__last2min', 'B_12__last__log', 'B_12__max', 'B_12__mean', 'B_12__mean2std', 'B_12__mean__log', 'B_12__min', 'B_12__range', 'B_12__std', 'B_13__first', 'B_13__last', 'B_13__last2max', 'B_13__last2min', 'B_13__last__log', 'B_13__max', 'B_13__mean', 'B_13__mean2std', 'B_13__mean__log', 'B_13__min', 'B_13__range', 'B_13__std', 'B_14__first', 'B_14__last', 'B_14__last2max', 'B_14__last2min', 'B_14__max', 'B_14__mean', 'B_14__mean2std', 'B_14__min', 'B_14__range', 'B_14__std', 'B_15__first', 'B_15__last', 'B_15__last2max', 'B_15__last2min', 'B_15__max', 'B_15__mean', 'B_15__mean2std', 'B_15__min', 'B_15__

2122

In [5]:
id_feats = ['customer_ID']
date_col =  'S_2'
target_col = 'target'

# x_cols = list(set(all_cols)-set(['customer_ID', 'S_2', 'target']))
# len(x_cols)

x_cols = ['P_2__last', 'P_2__min', 'P_2__mean', 'D_48__last', 'P_2__max', 'R_1__mean__log', 'D_48__mean', 'R_1__std', 'D_44__mean__log', 'B_2__last__log', 'B_2__last', 'R_1__max', 'P_2__first', 'D_44__last__log', 'R_1__range', 'B_18__last', 'B_9__last', 'B_3__last__log', 'D_61__last', 'B_2__mean', 'D_48__max', 'D_55__last', 'B_9__mean', 'B_3__mean__log', 'R_1__mean', 'B_33__last', 'D_44__last', 'D_44__max', 'B_18__mean', 'D_48__min', 'B_23__last__log', 'B_33__mean', 'B_3__last', 'B_33__last__log', 'B_7__last', 'B_11__last__log', 'D_75__last', 'D_58__last', 'B_2__last2max', 'B_20__last__log', 'B_23__last', 'R_10__mean__log', 'B_23__mean__log', 'B_2__mean__log', 'D_55__max', 'D_61__mean', 'B_3__max', 'B_18__last__log', 'B_22__mean__log', 'D_75__mean__log', 'B_2__min', 'B_9__last__log', 'D_75__last__log', 'D_44__mean', 'B_11__mean__log', 'R_1__last', 'B_20__mean__log', 'D_75__max', 'R_10__std', 'B_18__min', 'B_16__last', 'D_61__min', 'D_78__mean__log', 'D_48__first', 'B_9__mean__log', 'D_58__max', 'B_4__last', 'D_44__std', 'B_3__range', 'B_3__std', 'D_74__last', 'B_20__last', 'B_19__last__log', 'R_10__max', 'D_75__mean', 'R_10__range', 'D_44__range', 'B_19__mean__log', 'D_58__mean', 'B_22__max', 'B_19__last', 'D_74__mean__log', 'B_7__max', 'D_74__last__log', 'B_1__last', 'B_7__mean', 'B_23__max', 'B_37__last', 'R_1__last__log', 'B_4__last__log', 'R_2__mean__log', 'D_58__last__log', 'R_2__max', 'B_19__max', 'B_18__mean__log', 'B_3__mean', 'B_23__mean', 'R_2__std', 'B_33__mean__log', 'D_55__mean', 'D_74__max', 'B_40__last__log', 'D_58__mean__log', 'B_9__max', 'R_2__range', 'B_18__max', 'B_40__mean__log', 'B_11__last', 'B_4__mean__log', 'B_9__min', 'B_20__max', 'D_78__std', 'B_30=0.0', 'D_74__mean', 'B_22__mean', 'B_30__nunique', 'B_16__max', 'R_2__last', 'D_78__range', 'B_33__last2max', 'D_78__max', 'D_55__std', 'B_22__std', 'R_4__std', 'D_55__range', 'R_5__mean__log', 'B_18__last2max', 'D_55__last__log', 'R_4__mean__log', 'R_4__max', 'D_77__mean__log', 'B_22__range', 'B_1__mean', 'B_22__last', 'B_16__last__log', 'B_4__mean', 'R_4__range', 'B_37__mean', 'R_10__mean', 'R_8__mean__log', 'B_33__min', 'B_4__max', 'D_77__last__log', 'B_30=1.0', 'D_84__mean__log', 'D_75__min', 'B_4__last2max', 'B_19__mean', 'B_22__last__log', 'B_28__last2max', 'D_39__max', 'R_6__mean__log', 'D_41__range', 'B_11__mean', 'D_39__std', 'D_41__std', 'R_2__mean', 'D_58__min', 'B_14__last2max', 'D_39__range', 'B_19__range', 'S_7__max', 'D_55__mean__log', 'B_16__mean', 'B_7__min', 'B_11__last2max', 'B_2__max', 'S_7__mean', 'S_25__std', 'D_41__last', 'B_20__mean', 'D_41__max', 'B_1__max', 'B_37__max', 'S_3__max', 'B_16__last2max', 'S_25__range', 'D_44__first', 'P_3__min', 'S_3__mean', 'D_75__first', 'D_75__last2max', 'R_7__mean__log', 'B_9__first', 'P_3__mean', 'B_19__std', 'B_23__min', 'D_41__last__log', 'S_25__min', 'D_112__last__log', 'D_70__max', 'D_41__mean__log', 'R_4__mean', 'B_16__mean__log', 'D_74__min', 'B_18__first', 'B_33__max', 'B_11__max', 'D_70__mean__log', 'P_2__range', 'B_9__last2max', 'D_44__min', 'D_62__mean', 'D_78__last__log', 'P_2__std', 'D_70__std', 'B_6__last2max', 'B_4__min', 'S_7__last', 'S_25__mean', 'R_4__last', 'R_2__last__log', 'D_65__mean__log', 'B_23__range', 'B_2__first', 'B_38=2.0', 'D_112__last', 'D_62__max', 'R_3__mean', 'R_3__max', 'D_74__first', 'B_10__min', 'D_58__first', 'D_75__std', 'B_17__min', 'D_70__mean', 'B_25__last2max', 'D_78__last', 'D_70__range', 'B_20__last2max', 'D_75__range', 'B_23__last2max', 'P_3__last', 'S_3__last', 'D_112__last2max', 'B_8__min', 'D_55__min', 'D_39__last', 'D_62__last', 'D_78__mean', 'D_74__std', 'B_7__range', 'D_77__mean', 'S_15__max', 'R_8__std', 'B_37__last2max', 'B_23__std', 'D_74__range', 'B_38=4.0', 'B_1__min', 'B_8__mean', 'D_45__mean__log', 'S_7__range', 'D_58__last2max', 'B_7__first', 'R_8__max', 'R_8__range', 'D_77__max', 'B_8__first', 'R_15__std', 'D_70__last', 'B_37__min', 'R_15__max', 'D_77__last', 'R_15__range', 'S_7__std', 'B_7__std', 'D_74__last2max', 'B_23__first', 'R_15__mean__log', 'B_38__nunique', 'S_3__range', 'B_3__min', 'R_3__mean__log', 'D_43__mean', 'D_53__last__log', 'R_5__std', 'R_5__max', 'D_58__range', 'D_58__std', 'B_1__last2max', 'R_5__range', 'R_3__range', 'D_77__first', 'D_84__max', 'D_62__first', 'R_3__std', 'B_11__min', 'S_3__std', 'D_84__range', 'B_7__last2max', 'B_8__mean__log', 'B_3__last2max', 'B_28__mean__log', 'B_16__min', 'B_8__last', 'D_77__min', 'D_51__mean__log', 'D_84__std', 'D_70__last__log', 'B_33__first', 'D_45__last__log', 'S_15__range', 'B_38=5.0', 'B_20__range', 'D_43__max', 'D_53__mean__log', 'S_15__std', 'S_7__min', 'D_44__last2max', 'D_55__last2max', 'B_8__last__log', 'B_8__max', 'D_43__mean__log', 'D_62__min', 'D_39__mean', 'D_55__first', 'P_4__first', 'P_4__max', 'D_72__mean__log', 'B_3__first', 'D_84__last__log', 'R_15__mean', 'R_10__last', 'S_7__first', 'B_4__first', 'S_3__min', 'D_112__min', 'S_3__first', 'D_47__min', 'P_3__std', 'D_47__last', 'D_41__mean', 'D_39__mean__log', 'D_47__mean', 'P_4__mean__log', 'B_4__std', 'S_25__last', 'B_26__mean__log', 'D_81__mean__log', 'B_19__min', 'R_6__last__log', 'D_43__last', 'R_8__mean', 'B_10__last2max', 'D_47__first', 'B_9__std', 'B_20__std', 'B_1__first', 'D_45__first', 'D_45__max', 'R_5__last__log', 'B_40__min', 'D_43__last__log', 'B_37__first', 'D_47__max', 'R_16__mean__log', 'B_20__min', 'R_24__max', 'R_16__std', 'R_3__first', 'B_40__last2max', 'D_51__last__log', 'D_53__max', 'R_16__mean', 'D_53__last', 'D_54__last', 'R_24__range', 'R_24__std', 'D_54__last2max', 'D_65__last__log', 'B_4__range', 'D_43__std', 'B_19__first', 'D_45__mean', 'R_4__last__log', 'D_51__mean', 'D_39__last__log', 'R_3__last', 'B_9__range', 'R_20__mean__log', 'R_24__mean__log', 'R_10__last__log', 'B_28__last', 'R_27__last', 'R_27__min', 'D_51__last', 'D_84__last', 'R_5__mean', 'D_43__range', 'D_43__min', 'R_27__mean', 'R_14__mean__log', 'D_53__range', 'B_11__first', 'D_54__min', 'D_51__max', 'D_53__std', 'R_16__max', 'R_7__last__log', 'D_45__min', 'R_16__range', 'P_4__mean', 'B_22__first', 'D_45__last', 'D_79__mean__log', 'D_131__max', 'R_26__mean2std', 'B_16__std', 'D_54__range', 'D_131__mean__log', 'D_81__max', 'B_26__last__log', 'B_20__first', 'D_81__last', 'R_1__first', 'D_84__mean', 'B_19__last2max', 'D_72__max', 'D_131__last', 'S_15__mean', 'R_27__max', 'R_27__first', 'P_3__range', 'R_5__last', 'D_42__max', 'B_1__range', 'D_53__mean', 'D_112__mean', 'D_42__first', 'B_30=2.0', 'D_42__mean', 'B_37__range', 'B_11__range', 'B_17__mean2std', 'B_16__first', 'D_120__nunique', 'B_16__range', 'D_72__range', 'D_81__range', 'D_70__first', 'D_43__first', 'B_1__std', 'B_37__std', 'D_131__mean', 'D_52__min', 'D_89__mean__log', 'D_79__max', 'D_42__min', 'B_11__std', 'D_42__last', 'D_72__std', 'D_81__last__log', 'B_17__mean', 'D_89__std', 'D_81__std', 'D_54__std', 'D_89__max', 'R_13__max', 'D_89__range', 'R_8__last__log', 'R_13__std', 'D_79__last', 'D_120=0.0', 'D_112__range', 'D_72__last', 'R_13__range', 'D_52__mean', 'D_51__first', 'D_79__mean', 'D_72__last__log', 'R_3__last__log', 'S_8__min', 'D_106__mean2std', 'D_112__std', 'B_21__mean__log', 'R_8__last', 'B_28__min', 'D_130__last', 'D_42__mean2std', 'R_3__min', 'D_49__mean2std', 'D_52__last', 'R_24__mean', 'D_46__mean', 'S_25__last2max', 'D_130__max', 'D_62__last2max', 'D_120=1.0', 'D_127__max', 'D_127__mean__log', 'D_52__first', 'D_79__last__log', 'R_11__mean', 'D_131__last__log', 'D_51__min', 'B_25__last', 'D_130__mean__log', 'B_28__mean', 'D_53__mean2std', 'R_21__max', 'D_64=O', 'R_21__mean__log', 'S_11__min', 'S_16__mean__log', 'B_8__last2max', 'R_21__std', 'D_114=1.0', 'R_11__std', 'D_59__last', 'D_130__last__log', 'R_24__last', 'R_13__mean__log', 'D_130__mean', 'R_21__range', 'P_4__range', 'R_6__max', 'S_15__last', 'D_131__first', 'D_59__mean', 'D_133__mean__log', 'R_6__range', 'R_9__mean2std', 'D_81__mean', 'B_2__std', 'R_20__max', 'D_127__mean', 'D_52__max', 'D_70__min', 'R_21__mean', 'P_4__std', 'R_20__range', 'R_20__std', 'B_12__mean__log', 'B_25__min', 'D_135__mean2std', 'D_71__last__log', 'D_59__max', 'D_89__mean', 'D_137__mean2std', 'D_138__mean2std', 'D_72__mean', 'D_79__first', 'R_6__std', 'D_53__min', 'D_136__mean2std', 'D_132__mean2std', 'R_15__last', 'S_5__mean__log', 'D_112__mean__log', 'D_41__mean2std', 'D_116__nunique', 'D_134__mean2std', 'S_13__last2max', 'D_128__min', 'D_128__first', 'D_131__min', 'D_64__last=O', 'S_20__max', 'D_128__mean', 'D_51__std', 'R_11__mean__log', 'S_27__last__log', 'D_115__mean__log', 'S_27__last2max', 'P_3__first', 'B_2__range', 'S_8__last', 'D_128__mean__log', 'D_68=6.0', 'R_13__mean', 'R_11__max', 'D_79__std', 'B_12__last__log', 'S_20__std', 'D_127__last', 'D_128__max', 'D_79__min', 'B_25__mean', 'S_20__mean__log', 'D_128__last', 'D_59__min', 'R_11__range', 'D_53__first', 'D_127__range', 'S_20__range', 'D_118__max', 'D_115__mean', 'D_51__range', 'D_130__min', 'D_79__range', 'D_115__max', 'D_62__std', 'D_71__mean__log', 'D_62__range', 'D_115__last', 'D_119__max', 'D_68__nunique', 'B_20__mean2std', 'D_78__first', 'R_17__max', 'D_130__first', 'D_128__last__log', 'B_38=6.0', 'D_127__std', 'D_115__last__log', 'D_92__mean__log', 'B_17__last', 'R_17__std', 'B_24__mean__log', 'R_17__range', 'S_8__mean', 'R_12__last', 'D_118__mean', 'B_32__max', 'B_22__min', 'S_6__range', 'R_20__mean', 'R_12__min', 'D_119__mean', 'P_4__min', 'R_6__mean', 'B_32__range', 'D_118__mean__log', 'D_115__first', 'S_20__mean', 'D_48__std', 'D_118__last', 'S_8__last2max', 'P_4__last', 'B_28__max', 'D_115__min', 'D_119__last', 'D_119__mean__log', 'S_13__min', 'D_56__mean2std', 'D_54__mean', 'S_5__last__log', 'D_92__max', 'S_15__first', 'D_39__mean2std', 'B_32__std', 'S_6__last2max', 'S_22__max', 'D_121__max', 'S_6__std', 'D_121__last', 'D_118__first', 'R_6__last', 'R_12__range', 'D_133__last__log', 'D_121__mean', 'D_89__last', 'B_32__mean__log', 'D_127__first', 'D_119__first', 'D_122__mean', 'R_12__std', 'R_13__last', 'D_92__mean']
len(x_cols)

660

In [6]:
%%time
df = df[x_cols + [target_col]].copy(deep=True)

Wall time: 795 ms


## spFSR feature selection

In [7]:
import pandas as pd
import numpy as np
import os 
from pathlib import Path

from datetime import datetime, timedelta
import time 
from dateutil.relativedelta import relativedelta

from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [8]:
import pyarrow.parquet as pq
import pyarrow as pa

In [9]:
import warnings
warnings.filterwarnings("ignore")

pd.options.display.max_rows=999
pd.options.display.max_columns=999

In [10]:
df[x_cols].isna().sum()[df[x_cols].isna().sum()>0]

Series([], dtype: int64)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 458913 entries, 0 to 458912
Columns: 661 entries, P_2__last to target
dtypes: float32(654), int32(6), uint8(1)
memory usage: 1.1 GB


In [12]:
%%time
float64_cols = df.select_dtypes(include=['float64', 'float32']).columns.tolist()
df[float64_cols] = np.float16(df[float64_cols].values)
#---convert int64 to int32
int64_cols = df.select_dtypes(include=['int64', 'int32']).columns.tolist()
df[int64_cols] = np.int16(df[int64_cols].values)

Wall time: 1min 38s


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 458913 entries, 0 to 458912
Columns: 661 entries, P_2__last to target
dtypes: float16(654), int16(6), uint8(1)
memory usage: 581.6 MB


In [14]:
    
def save_log(log_file, log_message, verbose=False):
    f = open(log_file, "a")
    f.write(log_message)
    f.close()
    
    if verbose:
        print(log_message)

In [15]:
log_file='amex/spfsr-rev3data-top660corrfeats.txt'

In [16]:

from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from spFSR import SpFSR


In [17]:
25000*50

1250000

In [ ]:
n_samples = 25000

for seed in range(1, 50):
    df_ = df.sample(n=n_samples, replace=False, random_state=seed)
    x=df_[x_cols]
    y=df_['target']
    
    
    # set the engine parameters
    #sp_engine = SpFSR(x.values, y.values, pred_type='r', scoring='r2', wrapper=DecisionTreeRegressor(random_state=1))
#     wrapper = DecisionTreeClassifier(max_depth=20, random_state=seed)
#     wrapper = LogisticRegression(random_state=seed)
    wrapper = SVC(random_state=seed)
    
    sp_engine = SpFSR(x.values, y.values, pred_type='c', scoring='roc_auc', wrapper=wrapper)

    sp_run = sp_engine.run(num_features=200, n_samples_max=n_samples, print_freq=5, iter_max=15)  

    sp_results = sp_run.results

    print('Best value:', sp_results.get('selected_ft_score_mean'))
    print('Indices of selected features: ', sp_results.get('selected_features'))
    print('Importance of selected features: ', sp_results.get('selected_ft_importance').round(3))
    print('Total iterations for the optimal feature set:', sp_results.get('total_iter_for_opt'))

    sel_feats=list(np.array(x_cols)[sp_results.get('selected_features')])
    print(sel_feats)

In [20]:
n_samples = 25000

for seed in range(1, 50):
    df_ = df.sample(n=n_samples, replace=False, random_state=seed)
    x=df_[x_cols]
    y=df_['target']
    
    
    # set the engine parameters
    #sp_engine = SpFSR(x.values, y.values, pred_type='r', scoring='r2', wrapper=DecisionTreeRegressor(random_state=1))
#     wrapper = DecisionTreeClassifier(max_depth=20, random_state=seed)
    wrapper = LogisticRegression(random_state=seed)
#     wrapper = SVC(random_state=seed)
    
    sp_engine = SpFSR(x.values, y.values, pred_type='c', scoring='roc_auc', wrapper=wrapper)

    sp_run = sp_engine.run(num_features=0, n_samples_max=n_samples, print_freq=5, iter_max=25)  

    sp_results = sp_run.results

    print('Best value:', sp_results.get('selected_ft_score_mean'))
    print('Indices of selected features: ', sp_results.get('selected_features'))
    print('Importance of selected features: ', sp_results.get('selected_ft_importance').round(3))
    print('Total iterations for the optimal feature set:', sp_results.get('total_iter_for_opt'))

    sel_feats=list(np.array(x_cols)[sp_results.get('selected_features')])
    print(sel_feats)
    save_log(log_file, f'{sel_feats}\n\n')

SpFSR-INFO: Wrapper: LogisticRegression(random_state=1)
SpFSR-INFO: Hot start: True
SpFSR-INFO: Hot start range: 0.2
SpFSR-INFO: Feature weighting: False
SpFSR-INFO: Scoring metric: roc_auc
SpFSR-INFO: Number of jobs: 1
SpFSR-INFO: Number of observations in the dataset: 25000
SpFSR-INFO: Number of observations used: 25000
SpFSR-INFO: Number of features available: 660
SpFSR-INFO: Number of features to select: 0
SpFSR-INFO: iter_no: 0, num_ft: 9, value: 0.936, st_dev: 0.003, best: 0.936 @ iter_no 0
SpFSR-INFO: iter_no: 5, num_ft: 12, value: 0.938, st_dev: 0.002, best: 0.938 @ iter_no 5
SpFSR-INFO: iter_no: 10, num_ft: 24, value: 0.942, st_dev: 0.002, best: 0.942 @ iter_no 8
SpFSR-INFO: iter_no: 15, num_ft: 29, value: 0.943, st_dev: 0.003, best: 0.943 @ iter_no 13
SpFSR-INFO: ===> iter_no: 18, same feature stall limit reached, initializing search...
SpFSR-INFO: iter_no: 20, num_ft: 7, value: 0.936, st_dev: 0.004, best: 0.943 @ iter_no 13
SpFSR-INFO: ===> iter_no: 22, same feature stall li

Best value: 0.943
Indices of selected features:  [2, 26, 39, 9, 11, 0, 450, 51, 5, 15, 187, 103, 133, 25, 38, 3, 84, 4, 425, 7, 107, 89, 52, 88, 87]
Importance of selected features:  [0.156 0.121 0.114 0.105 0.102 0.1   0.087 0.085 0.072 0.071 0.07  0.068
 0.063 0.056 0.056 0.045 0.042 0.041 0.04  0.035 0.034 0.032 0.019 0.013
 0.013]
Total iterations for the optimal feature set: 13
['P_2__mean', 'D_44__last', 'B_20__last__log', 'B_2__last__log', 'R_1__max', 'P_2__last', 'D_42__last', 'B_9__last__log', 'R_1__mean__log', 'B_18__last', 'D_41__last__log', 'B_9__max', 'B_22__last', 'B_33__last', 'B_2__last2max', 'D_48__last', 'B_1__last', 'P_2__max', 'D_42__max', 'R_1__std', 'B_11__last', 'B_4__last__log', 'D_75__last__log', 'R_1__last__log', 'B_37__last']


SpFSR-INFO: Wrapper: LogisticRegression(random_state=2)
SpFSR-INFO: Hot start: True
SpFSR-INFO: Hot start range: 0.2
SpFSR-INFO: Feature weighting: False
SpFSR-INFO: Scoring metric: roc_auc
SpFSR-INFO: Number of jobs: 1
SpFSR-INFO: Number of observations in the dataset: 25000
SpFSR-INFO: Number of observations used: 25000
SpFSR-INFO: Number of features available: 660
SpFSR-INFO: Number of features to select: 0
SpFSR-INFO: iter_no: 0, num_ft: 8, value: 0.935, st_dev: 0.003, best: 0.935 @ iter_no 0
SpFSR-INFO: iter_no: 5, num_ft: 11, value: 0.936, st_dev: 0.003, best: 0.936 @ iter_no 2
SpFSR-INFO: iter_no: 10, num_ft: 27, value: 0.942, st_dev: 0.003, best: 0.942 @ iter_no 7
SpFSR-INFO: iter_no: 15, num_ft: 29, value: 0.944, st_dev: 0.003, best: 0.944 @ iter_no 12
SpFSR-INFO: iter_no: 20, num_ft: 28, value: 0.944, st_dev: 0.004, best: 0.944 @ iter_no 12
SpFSR-INFO: iter_no: 25, num_ft: 28, value: 0.944, st_dev: 0.003, best: 0.944 @ iter_no 12
SpFSR-INFO: SpFSR completed in 2.24 minutes.
S

Best value: 0.944
Indices of selected features:  [2, 9, 15, 169, 26, 41, 39, 7, 184, 3, 55, 16, 35, 450, 4, 72, 38, 232, 107, 137, 0, 211, 11, 429, 49, 349, 27, 370]
Importance of selected features:  [0.321 0.178 0.17  0.156 0.134 0.131 0.113 0.107 0.104 0.095 0.093 0.081
 0.08  0.075 0.072 0.072 0.072 0.063 0.062 0.051 0.046 0.038 0.034 0.027
 0.026 0.019 0.017 0.005]
Total iterations for the optimal feature set: 12
['P_2__mean', 'B_2__last__log', 'B_18__last', 'D_41__last', 'D_44__last', 'R_10__mean__log', 'B_20__last__log', 'R_1__std', 'P_3__mean', 'D_48__last', 'R_1__last', 'B_9__last', 'B_11__last__log', 'D_42__last', 'P_2__max', 'B_19__last__log', 'B_2__last2max', 'P_3__last', 'B_11__last', 'B_37__mean', 'P_2__last', 'R_2__last__log', 'R_1__max', 'D_42__first', 'D_75__mean__log', 'D_45__first', 'D_44__max', 'D_65__last__log']


SpFSR-INFO: Wrapper: LogisticRegression(random_state=3)
SpFSR-INFO: Hot start: True
SpFSR-INFO: Hot start range: 0.2
SpFSR-INFO: Feature weighting: False
SpFSR-INFO: Scoring metric: roc_auc
SpFSR-INFO: Number of jobs: 1
SpFSR-INFO: Number of observations in the dataset: 25000
SpFSR-INFO: Number of observations used: 25000
SpFSR-INFO: Number of features available: 660
SpFSR-INFO: Number of features to select: 0
SpFSR-INFO: iter_no: 0, num_ft: 9, value: 0.935, st_dev: 0.003, best: 0.935 @ iter_no 0
SpFSR-INFO: iter_no: 5, num_ft: 12, value: 0.939, st_dev: 0.004, best: 0.939 @ iter_no 5
SpFSR-INFO: iter_no: 10, num_ft: 25, value: 0.943, st_dev: 0.003, best: 0.943 @ iter_no 8
SpFSR-INFO: iter_no: 15, num_ft: 23, value: 0.943, st_dev: 0.002, best: 0.943 @ iter_no 8
SpFSR-INFO: ===> iter_no: 15, same feature stall limit reached, initializing search...
SpFSR-INFO: iter_no: 20, num_ft: 9, value: 0.935, st_dev: 0.003, best: 0.943 @ iter_no 8
SpFSR-INFO: iter_no: 25, num_ft: 18, value: 0.942, st

...

Best value: 0.948
Indices of selected features:  [10, 2, 0, 39, 237, 137, 429, 55, 35, 277, 88, 38, 1, 273, 208, 8, 4, 89, 17, 34, 11, 425, 9, 15, 107, 26, 87, 53, 32]
Importance of selected features:  [0.232 0.196 0.159 0.133 0.121 0.117 0.111 0.099 0.097 0.079 0.075 0.067
 0.066 0.063 0.063 0.054 0.051 0.049 0.049 0.045 0.042 0.037 0.035 0.03
 0.029 0.016 0.013 0.    0.   ]
Total iterations for the optimal feature set: 22
['B_2__last', 'P_2__mean', 'P_2__last', 'B_20__last__log', 'D_39__last', 'B_37__mean', 'D_42__first', 'R_1__last', 'B_11__last__log', 'R_5__max', 'R_1__last__log', 'B_2__last2max', 'P_2__min', 'R_3__mean__log', 'S_7__last', 'D_44__mean__log', 'P_2__max', 'B_4__last__log', 'B_3__last__log', 'B_7__last', 'R_1__max', 'D_42__max', 'B_2__last__log', 'B_18__last', 'B_11__last', 'D_44__last', 'B_37__last', 'D_44__mean', 'B_3__last']


SpFSR-INFO: Wrapper: LogisticRegression(random_state=48)
SpFSR-INFO: Hot start: True
SpFSR-INFO: Hot start range: 0.2
SpFSR-INFO: Feature weighting: False
SpFSR-INFO: Scoring metric: roc_auc
SpFSR-INFO: Number of jobs: 1
SpFSR-INFO: Number of observations in the dataset: 25000
SpFSR-INFO: Number of observations used: 25000
SpFSR-INFO: Number of features available: 660
SpFSR-INFO: Number of features to select: 0
SpFSR-INFO: iter_no: 0, num_ft: 8, value: 0.938, st_dev: 0.002, best: 0.938 @ iter_no 0
SpFSR-INFO: iter_no: 5, num_ft: 21, value: 0.948, st_dev: 0.002, best: 0.948 @ iter_no 5
SpFSR-INFO: iter_no: 10, num_ft: 28, value: 0.95, st_dev: 0.003, best: 0.95 @ iter_no 10
SpFSR-INFO: iter_no: 15, num_ft: 29, value: 0.95, st_dev: 0.003, best: 0.95 @ iter_no 10
SpFSR-INFO: iter_no: 20, num_ft: 29, value: 0.95, st_dev: 0.003, best: 0.95 @ iter_no 10
SpFSR-INFO: iter_no: 25, num_ft: 31, value: 0.95, st_dev: 0.003, best: 0.95 @ iter_no 10
SpFSR-INFO: SpFSR completed in 2.31 minutes.
SpFSR-I

Best value: 0.95
Indices of selected features:  [0, 1, 2, 7, 26, 233, 349, 9, 15, 6, 425, 12, 412, 169, 51, 212, 3, 32, 64, 87, 377, 159, 5, 192, 158, 117, 17, 107]
Importance of selected features:  [0.268 0.202 0.164 0.123 0.123 0.118 0.107 0.09  0.078 0.077 0.071 0.057
 0.055 0.053 0.053 0.043 0.043 0.042 0.036 0.036 0.03  0.025 0.019 0.012
 0.009 0.004 0.003 0.001]
Total iterations for the optimal feature set: 10
['P_2__last', 'P_2__min', 'P_2__mean', 'R_1__std', 'D_44__last', 'S_3__last', 'D_45__first', 'B_2__last__log', 'B_18__last', 'D_48__mean', 'D_42__max', 'P_2__first', 'B_26__last__log', 'D_41__last', 'B_9__last__log', 'D_65__mean__log', 'D_48__last', 'B_3__last', 'B_9__mean__log', 'B_37__last', 'D_39__last__log', 'D_39__range', 'R_1__mean__log', 'R_4__mean', 'B_14__last2max', 'R_2__last', 'B_3__last__log', 'B_11__last']


SpFSR-INFO: Wrapper: LogisticRegression(random_state=49)
SpFSR-INFO: Hot start: True
SpFSR-INFO: Hot start range: 0.2
SpFSR-INFO: Feature weighting: False
SpFSR-INFO: Scoring metric: roc_auc
SpFSR-INFO: Number of jobs: 1
SpFSR-INFO: Number of observations in the dataset: 25000
SpFSR-INFO: Number of observations used: 25000
SpFSR-INFO: Number of features available: 660
SpFSR-INFO: Number of features to select: 0
SpFSR-INFO: iter_no: 0, num_ft: 9, value: 0.937, st_dev: 0.003, best: 0.937 @ iter_no 0
SpFSR-INFO: iter_no: 5, num_ft: 21, value: 0.945, st_dev: 0.002, best: 0.945 @ iter_no 4
SpFSR-INFO: iter_no: 10, num_ft: 27, value: 0.947, st_dev: 0.003, best: 0.947 @ iter_no 8
SpFSR-INFO: iter_no: 15, num_ft: 27, value: 0.947, st_dev: 0.002, best: 0.947 @ iter_no 8
SpFSR-INFO: iter_no: 20, num_ft: 28, value: 0.948, st_dev: 0.004, best: 0.948 @ iter_no 16
SpFSR-INFO: iter_no: 25, num_ft: 31, value: 0.948, st_dev: 0.002, best: 0.948 @ iter_no 16
SpFSR-INFO: SpFSR completed in 2.29 minutes.
S

Best value: 0.948
Indices of selected features:  [15, 9, 233, 107, 49, 51, 3, 12, 39, 219, 26, 69, 117, 0, 88, 450, 377, 429, 19, 208, 64, 137, 55, 5, 27, 274, 4, 1]
Importance of selected features:  [0.203 0.186 0.175 0.129 0.128 0.125 0.118 0.113 0.105 0.103 0.085 0.077
 0.076 0.074 0.069 0.069 0.065 0.063 0.041 0.041 0.036 0.025 0.021 0.017
 0.006 0.005 0.005 0.004]
Total iterations for the optimal feature set: 16
['B_18__last', 'B_2__last__log', 'S_3__last', 'B_11__last', 'D_75__mean__log', 'B_9__last__log', 'D_48__last', 'P_2__first', 'B_20__last__log', 'R_3__max', 'D_44__last', 'B_3__std', 'R_2__last', 'P_2__last', 'R_1__last__log', 'D_42__last', 'D_39__last__log', 'D_42__first', 'B_2__mean', 'S_7__last', 'B_9__mean__log', 'B_37__mean', 'R_1__last', 'R_1__mean__log', 'D_44__max', 'D_43__mean', 'P_2__max', 'P_2__min']


In [ ]:
n_samples = 45000

for seed in range(20):
    df_ = df.sample(n=n_samples, replace=False, random_state=seed)
    x=df_[x_cols]
    y=df_['target']
    
    
    # set the engine parameters
    #sp_engine = SpFSR(x.values, y.values, pred_type='r', scoring='r2', wrapper=DecisionTreeRegressor(random_state=1))
    sp_engine = SpFSR(x.values, y.values, pred_type='c', scoring='roc_auc', wrapper=DecisionTreeClassifier(random_state=1))

    sp_run = sp_engine.run(num_features=100, n_samples_max=n_samples, print_freq=5, iter_max=20)  

    sp_results = sp_run.results

    print('Best value:', sp_results.get('selected_ft_score_mean'))
    print('Indices of selected features: ', sp_results.get('selected_features'))
    print('Importance of selected features: ', sp_results.get('selected_ft_importance').round(3))
    print('Total iterations for the optimal feature set:', sp_results.get('total_iter_for_opt'))

    sel_feats=list(np.array(x_cols)[sp_results.get('selected_features')])
    print(sel_feats)

In [ ]:
n_samples = 45000

for seed in range(20):
    df_ = df.sample(n=n_samples, replace=False, random_state=seed)
    x=df_[x_cols]
    y=df_['target']
    
    
    # set the engine parameters
    #sp_engine = SpFSR(x.values, y.values, pred_type='r', scoring='r2', wrapper=DecisionTreeRegressor(random_state=1))
    sp_engine = SpFSR(x.values, y.values, pred_type='c', scoring='roc_auc', wrapper=DecisionTreeClassifier(random_state=1))

    sp_run = sp_engine.run(num_features=100, n_samples_max=n_samples, print_freq=5, iter_max=20)  

    sp_results = sp_run.results

    print('Best value:', sp_results.get('selected_ft_score_mean'))
    print('Indices of selected features: ', sp_results.get('selected_features'))
    print('Importance of selected features: ', sp_results.get('selected_ft_importance').round(3))
    print('Total iterations for the optimal feature set:', sp_results.get('total_iter_for_opt'))

    sel_feats=list(np.array(x_cols)[sp_results.get('selected_features')])
    print(sel_feats)

In [ ]:
n_samples = 45000

for seed in range(20):
    df_ = df.sample(n=n_samples, replace=False, random_state=seed)
    x=df_[x_cols]
    y=df_['target']
    
    
    # set the engine parameters
    #sp_engine = SpFSR(x.values, y.values, pred_type='r', scoring='r2', wrapper=DecisionTreeRegressor(random_state=1))
    sp_engine = SpFSR(x.values, y.values, pred_type='c', scoring='roc_auc', wrapper=DecisionTreeClassifier)

    sp_run = sp_engine.run(num_features=100, n_samples_max=n_samples, print_freq=5, iter_max=20)  

    sp_results = sp_run.results

    print('Best value:', sp_results.get('selected_ft_score_mean'))
    print('Indices of selected features: ', sp_results.get('selected_features'))
    print('Importance of selected features: ', sp_results.get('selected_ft_importance').round(3))
    print('Total iterations for the optimal feature set:', sp_results.get('total_iter_for_opt'))

    sel_feats=list(np.array(x_cols)[sp_results.get('selected_features')])
    print(sel_feats)